# Simple Ising Model Example

This notebook demonstrates:
1. Creating an Ising model (from SpinSystems submodule)
2. Setting up a Metropolis algorithm with Boltzmann weights
3. Configuring measurements with the new measurement framework
4. Running equilibrium Monte Carlo simulation
5. Analyzing results

In [1]:
using Pkg
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.instantiate()

using Random
using StatsBase
using MonteCarloX
using MonteCarloX.SpinSystems

  Activating project at `~/.julia/dev/MonteCarloX`
Precompiling packages...
   1256.1 ms  ✓ MonteCarloX
  1 dependency successfully precompiled in 3 seconds. 235 already precompiled.


## Setup System and Algorithm

In [2]:
# Setup
rng = MersenneTwister(42)
L = 8
# Create Ising system
sys = Ising([L, L], J=1, periodic=true)
init!(sys, :random, rng=rng)

println("System configuration:")
println("  Lattice size: $(L)×$(L)")
println("  Total spins: $(L*L)")
println("  Spins from system: $(size(sys.spins))")

# Create Metropolis algorithm with Boltzmann weight
β = 0.4  # Inverse temperature
alg = Metropolis(rng, β=β)
println("  Inverse temperature β: $β")

System configuration:
  Lattice size: 8×8
  Total spins: 64
  Spins from system: (64,)
  Inverse temperature β: 0.4


## Configure Measurements

In [3]:
# Setup measurements
measurements = Measurements([
    :energy => energy => Float64[],
    :magnetization => magnetization => Float64[]
], interval=10)

Measurements{Symbol, IntervalSchedule}(Dict{Symbol, Measurement}(:energy => Measurement{typeof(energy), Vector{Float64}}(MonteCarloX.SpinSystems.energy, Float64[]), :magnetization => Measurement{typeof(magnetization), Vector{Float64}}(MonteCarloX.SpinSystems.magnetization, Float64[])), IntervalSchedule(10.0, 0.0))

## Run Simulation

In [4]:
# Thermalization
println("Thermalizing...")
N = L * L
for i in 1:N*1000
    spin_flip!(sys, alg)
end
println("  Thermalization complete: $(alg.steps) steps")

# Reset statistics after thermalization
reset_statistics!(alg)

Thermalizing...
  Thermalization complete: 64000 steps


0

In [5]:
# Production run
println("Production run...")
for i in 1:N*10000
    spin_flip!(sys, alg)
    measure!(measurements, sys, i)
end

Production run...


## Analyze Results

In [6]:
energies = measurements[:energy].data
mags = measurements[:magnetization].data

avg_E = mean(energies) / N
avg_M = mean(mags) / N

println("\nResults:")
println("  Total steps: $(alg.steps)")
println("  Acceptance rate: $(round(acceptance_rate(alg), digits=4))")
println("\nObservables (per spin):")
println("  Average energy: $(round(avg_E, digits=4))")
println("  Average magnetization: $(round(avg_M, digits=4))")


Results:
  Total steps: 640000
  Acceptance rate: 0.2789

Observables (per spin):
  Average energy: -1.2182
  Average magnetization: 0.5995
